In [1]:
import nltk
from nltk import sent_tokenize,word_tokenize, pos_tag, ne_chunk
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk import wordpunct_tokenize
import re
from nltk.corpus import stopwords
import pandas as pd
from clean_data import process_sentence

##2017 12 3 using a different parser to parse sentence
'''
from nltk.parse.stanford import StanfordDependencyParser
path_to_jar = '/Users/collin/stanford/stanford-parser-full-2017-06-09/stanford-parser.jar'
path_to_models_jar = '/Users/collin/stanford/stanford-parser-full-2017-06-09/stanford-parser-3.8.0-models.jar'
dependency_parser = StanfordDependencyParser(path_to_jar=path_to_jar, path_to_models_jar=path_to_models_jar)
'''

from nltk.parse.corenlp import CoreNLPServer, CoreNLPDependencyParser
path_to_jar = '/Users/collin/stanford/stanford-corenlp-full-2017-06-09/stanford-corenlp-3.8.0.jar'
path_to_models_jar = '/Users/collin/stanford/stanford-corenlp-full-2017-06-09/stanford-corenlp-3.8.0-models.jar'
server = CoreNLPServer(path_to_jar=path_to_jar, path_to_models_jar=path_to_models_jar)
server.start()
dependency_parser = CoreNLPDependencyParser()

stemmer = SnowballStemmer('english')
def stem(w):
    return stemmer.stem(w)

DR_one = ['nsubj','dobj','xsubj','csubj','nmod','iobj']
DR_two = ['amod']
#DR_two = ['nsubj','dobj','xsubj','csubj','nsubjpass','nmod','iobj']
DR_three = ['conj']
DR = DR_one + DR_three

def extract_rule(dep_dic):
    value_list = []
    one_list = []
    three_list = []
    for key,value in dep_dic.items():
        if key in DR_one:
            one_list += value
        elif key in DR_two:
            three_list += value
    value_list.append(one_list)
    value_list.append(three_list)      
    return value_list

def dict_update(target_dict, new_w, word):
    if new_w in target_dict.keys():
        target_dict[new_w].append(word)
    else:
        target_dict[new_w] = [word]

def extract_target_phase(target_dict, s):
    ##parse the sentence
    result = dependency_parser.raw_parse(s)
    dep = next(result)
    dep_list = dict(sorted(dep.nodes.items()))

    for _, node in sorted(dep.nodes.items()):
         if node['word'] is not None:
            w = stem(node['word'])
            dep_dict = dict(node['deps'])
            if w in target_list and node['tag'][:2] == 'NN' and 'compound' in dep_dict.keys():
                new_w = ''
                word = ''
                adj_index = dep_dict['compound'][0] - 1
                adj_node = dep_list[adj_index]
                if adj_node['tag'][:2] == 'JJ' and adj_node['head'] == adj_index + 1:
                    new_w = stem(adj_node['word'])
                    word = adj_node['word']
                for index in dep_dict['compound']:
                    comp_word = dep_list[index]['word']
                    if new_w =='':
                        new_w = stem(comp_word)
                        word = comp_word
                    else:
                        new_w = new_w + ' ' + stem(comp_word)
                        word = word + ' ' + comp_word
                new_w = new_w + ' ' + w
                word = word + ' ' + node['word']
                dict_update(target_dict, new_w, word)
            elif w in target_list and node['tag'][:2] == 'NN' and node['rel'] !='compound':
                new_w = ''
                word = ''
                adj_index = node['address'] - 1
                adj_node = dep_list[adj_index]
                if adj_node['tag'][:2] == 'JJ' and adj_node['head'] == adj_index + 1:
                    new_w = stem(adj_node['word'])
                    word = adj_node['word']
                comp_word = node['word']
                if new_w =='':
                    new_w = stem(comp_word)
                    word = comp_word
                else:
                    new_w = new_w + ' ' + stem(comp_word)
                    word = word + ' ' + comp_word
                dict_update(target_dict, new_w, word)


def update_target(sents):
    target_d = {}
    sent_list = sent_tokenize(sents)
    for sent in sent_list:
        extract_target_phase(target_d,sent)
    for key in target_d.keys():
        target_d[key] = list(set(target_d[key]))
    return target_d


f1 = open('opinion_list_1.txt','r')
lines = f1.readlines()
opinion_list =[line.split("\n")[0] for line in lines]

f2 = open('target_list_1.txt','r')
lines = f2.readlines()
target_list = [line.split("\n")[0] for line in lines]

print(opinion_list)
print(target_list)

df = pd.read_csv('data.csv',index_col = 0)
df['comment'] = df['comment'].apply(process_sentence)
sents = df['comment'].values

size = len(df)


target_dict = []
for i in range(size):
    #print(i)
    dic = update_target(sents[i])
    target_dict.append(dic)

dict_df = {'comment':list(df['comment']), 'target':target_dict}
new_df = pd.DataFrame(data = dict_df)
new_df.head()
new_df.to_csv('transaction_v3.csv')

server.stop()
print("finish..")


['defect', 'perfect', 'hypocrit', 'nag', 'conserv', 'honesti', 'sneer', 'cashback', 'primit', 'cruel', 'delight', 'hoodium', 'blasphem', 'smuttiest', 'dilig', 'imperil', 'uncontrol', 'fanfar', 'smitten', 'outperform', 'portabl', 'sturdier', 'delirium', 'haughti', 'last-ditch', 'predica', 'rejoic', 'backach', 'illusori', 'denial', 'caustic', 'rapport', 'offenc', 'merrili', 'stellar', 'lame', 'desert', 'mania', 'goon', 'desir', 'cold', 'demolish', 'creepi', 'hegemoni', 'invect', 'uphold', 'fright', 'shiver', 'stoog', 'amiabili', 'sti', 'effigi', 'hot', 'lifeless', 'liar', 'inordin', 'inhibit', 'dent', 'retali', 'calm', 'exult', 'manipul', 'irret', 'overdo', 'random', 'right', 'shine', 'sadden', 'supremaci', 'strangl', 'approv', 'grudg', 'dirt', 'effus', 'obstruct', 'pieti', 'phobia', 'skeleton', 'acrimoni', 'fell', 'groundless', 'cri', 'kook', 'love', 'foremost', 'browbeat', 'rake', 'sucker', 'energet', 'illus', 'letch', 'freshest', 'sloth', 'effici', 'resound', 'contradict', 'defens', '

In [2]:
stem("digital")

'digit'

In [2]:
s = "cool toyyep this is my first digital camera , and what a ' toy ' it is ! i am a software engineer and am very keen into technical details of everything i buy , i spend around 3 months before buying the digital camera ; and i must say , g3 worth every single cent i spent on it just a little overview , powershot g3 is the flagship of canon 's powershot series and its an slr-like camera , its 4 megapixel and alsmost full manual control gives the pictures a touch of brilliance whether you are a novice or an expert , its ease of use and functionality goes together + you can have different kind of lens if you want + flashes , etc. as its 4mp , you might need bigger storage to store high quality images and recording movies you can record 3 minutes of video i am using kingston 512mb cf which works great and is very fast a good choice could be cf type ii , microdrives that can store around 1 gb of images some things that i did n't like but hey nothing is perfect . main dial is not backlit lens visible in optical viewfinder well thats only for old-school die-hard optical viewfinder fans use lcd instead which is brilliant and you can twist around too overall i 'm happy with my toy "

In [1]:
import nltk
from nltk import sent_tokenize,word_tokenize, pos_tag, ne_chunk
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk import wordpunct_tokenize
import re
from nltk.corpus import stopwords
import pandas as pd
from clean_data import process_sentence

##2017 12 3 using a different parser to parse sentence
'''
from nltk.parse.stanford import StanfordDependencyParser
path_to_jar = '/Users/collin/stanford/stanford-parser-full-2017-06-09/stanford-parser.jar'
path_to_models_jar = '/Users/collin/stanford/stanford-parser-full-2017-06-09/stanford-parser-3.8.0-models.jar'
dependency_parser = StanfordDependencyParser(path_to_jar=path_to_jar, path_to_models_jar=path_to_models_jar)
'''

from nltk.parse.corenlp import CoreNLPServer, CoreNLPDependencyParser
path_to_jar = '/Users/collin/stanford/stanford-corenlp-full-2017-06-09/stanford-corenlp-3.8.0.jar'
path_to_models_jar = '/Users/collin/stanford/stanford-corenlp-full-2017-06-09/stanford-corenlp-3.8.0-models.jar'
server = CoreNLPServer(path_to_jar=path_to_jar, path_to_models_jar=path_to_models_jar)
server.start()
dependency_parser = CoreNLPDependencyParser()

In [3]:
result = dependency_parser.raw_parse(s)
dep = next(result)
dep_list = dict(sorted(dep.nodes.items()))

In [4]:
print(dep_list)

{0: {'address': 0, 'word': None, 'lemma': None, 'ctag': 'TOP', 'tag': 'TOP', 'feats': None, 'head': None, 'deps': defaultdict(<class 'list'>, {'ROOT': [2]}), 'rel': None}, 1: {'address': 1, 'word': 'cool', 'lemma': 'cool', 'ctag': 'JJ', 'tag': 'JJ', 'feats': '_', 'head': 2, 'deps': defaultdict(<class 'list'>, {}), 'rel': 'amod'}, 2: {'address': 2, 'word': 'toyyep', 'lemma': 'toyyep', 'ctag': 'NN', 'tag': 'NN', 'feats': '_', 'head': 0, 'deps': defaultdict(<class 'list'>, {'amod': [1], 'acl:relcl': [8], 'punct': [9], 'cc': [10], 'conj': [11]}), 'rel': 'ROOT'}, 3: {'address': 3, 'word': 'this', 'lemma': 'this', 'ctag': 'DT', 'tag': 'DT', 'feats': '_', 'head': 8, 'deps': defaultdict(<class 'list'>, {}), 'rel': 'nsubj'}, 4: {'address': 4, 'word': 'is', 'lemma': 'be', 'ctag': 'VBZ', 'tag': 'VBZ', 'feats': '_', 'head': 8, 'deps': defaultdict(<class 'list'>, {}), 'rel': 'cop'}, 5: {'address': 5, 'word': 'my', 'lemma': 'my', 'ctag': 'PRP$', 'tag': 'PRP$', 'feats': '_', 'head': 8, 'deps': defaul

In [5]:
server.stop()